In [1]:
pip install --quiet azure-ai-projects==1.0.0b9 azure-identity azure-monitor-opentelemetry opentelemetry-sdk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true" # False by default

In [3]:
import dotenv
dotenv.load_dotenv()

True

In [5]:
print(f"{os.environ['PROJECT_CONNECTION_STRING']}")

southindia.api.azureml.ms;dfe81aa9-b256-4fe8-9cd5-23732f4d59c6;TechNTomorrow;tntproject


In [6]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [7]:

import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet

# Import your custom functions to be used as Tools for the Agent
from user_functions import user_functions

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

AGENT_NAME = "Seattle Tourist Assistant"

# Add Tools to be used by Agent
functions = FunctionTool(user_functions)

toolset = ToolSet()
toolset.add(functions)

# To enable tool calls executed automatically
project_client.agents.enable_auto_function_calls(toolset=toolset)

In [8]:
from azure.monitor.opentelemetry import configure_azure_monitor
connection_string = project_client.telemetry.get_connection_string()

if not connection_string:
    print("Application Insights is not enabled. Enable by going to Tracing in your Azure AI Foundry project.")
    exit()

configure_azure_monitor(connection_string=connection_string) #enable telemetry collection

In [9]:
from opentelemetry import trace
tracer = trace.get_tracer(__name__)

with tracer.start_as_current_span("seattle_agent-tracing"):
    agent = project_client.agents.create_agent(
    model=os.environ["AGENT_MODEL_DEPLOYMENT_NAME"],
    name=AGENT_NAME,
    instructions="You are a helpful assistant",
    toolset=toolset,
    )

    print(f"Created agent, ID: {agent.id}")

    # Create a thread for the agent to interact with
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread

    MESSAGE = "Can you email me weather info for Tokyo ?"

    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=MESSAGE,
    )
    print(f"Created message, ID: {message.id}")

    ### Execute the Agent run
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    print(f"Run ID: {run.id}")

Created agent, ID: asst_VcCpE3cOhtDmV1NqdVubdnsx
Created thread, ID: thread_2AHmTD8bg9y3CvLyO5Epr4S9
Created message, ID: msg_nrMRWVQmrTCSAUQOdHIR5M9s
Run finished with status: RunStatus.COMPLETED
Run ID: run_I0D807ofXbaSstNLP0ngQKsu


In [ ]:
from azure.monitor.opentelemetry import configure_azure_monitor
connection_string = project_client.telemetry.get_connection_string()

if not connection_string:
    print("Application Insights is not enabled. Enable by going to Tracing in your Azure AI Foundry project.")
    exit()

configure_azure_monitor(connection_string=connection_string) #enable telemetry collection

In [ ]:
from opentelemetry import trace
tracer = trace.get_tracer(__name__)

with tracer.start_as_current_span("example-tracing"):
    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="my-assistant",
        instructions="You are a helpful assistant"
    )
    thread = project_client.agents.create_thread()
    message = project_client.agents.create_message(
        thread_id=thread.id, role="user", content="Tell me a joke"
    )
    run = project_client.agents.create_run(thread_id=thread.id, agent_id=agent.id)